# Learning 4: GSD Fundamentals - Ground Sampling Distance for Drone Imaging

**Duration**: 20-25 minutes  
**Level**: Beginner  
**Prerequisites**: Basic optics knowledge (focal length, sensors)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Calculate** the Ground Sampling Distance (GSD) given altitude, focal length, and pixel pitch
2. **Plan missions** with specific GSD requirements
3. **Compare** different lens choices for drone imaging
4. **Understand** when diffraction limits matter vs when they don't
5. **Distinguish** between "pixel-limited" and "diffraction-limited" imaging

---

## Why GSD Matters

**Ground Sampling Distance (GSD)** is the real-world distance represented by one pixel in your drone imagery. It determines:

- What size features you can detect (cars, people, potholes)
- How much area you can cover per flight
- The accuracy of measurements from your imagery

Understanding GSD helps you:
- Choose the right altitude for your mission
- Select appropriate lenses and sensors
- Set realistic expectations for image resolution
- Balance coverage vs detail for your application

## Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# SPIDS imports
from prism.scenarios import get_scenario_preset, list_scenario_presets


# For interactive widgets (optional - graceful fallback if not available)
try:
    import ipywidgets as widgets
    from IPython.display import display

    WIDGETS_AVAILABLE = True
except ImportError:
    WIDGETS_AVAILABLE = False
    print("Note: ipywidgets not available. Interactive elements will use static displays.")

# Plotting style
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["font.size"] = 11

print("Setup complete!")

---

# Section 1: The GSD Formula

## The Geometry of Aerial Imaging

When a drone captures an image from altitude, each sensor pixel "sees" a small patch of ground. The size of this patch is the **Ground Sampling Distance (GSD)**:

$$\text{GSD} = \frac{H \times p}{f}$$

Where:
- $H$ = Flight altitude (meters)
- $p$ = Pixel pitch (sensor pixel size, in meters)
- $f$ = Focal length (meters)

### Intuition

- **Higher altitude** → Larger GSD (pixels cover more ground)
- **Larger pixels** → Larger GSD (each pixel sees more area)
- **Longer focal length** → Smaller GSD (more "zoom" = finer detail)

### Practical Example

A typical setup: **50mm lens, 6.5µm pixels, 100m altitude**

$$\text{GSD} = \frac{100 \times 6.5 \times 10^{-6}}{0.050} = 0.013 \text{ m} = 1.3 \text{ cm}$$

This means each pixel represents a 1.3cm × 1.3cm patch on the ground.

## Interactive GSD Calculator

Let's explore how altitude, focal length, and pixel pitch affect GSD:

In [ ]:
def calculate_gsd(altitude_m: float, focal_length_mm: float, pixel_pitch_um: float) -> dict:
    """
    Calculate Ground Sampling Distance and related parameters.

    Parameters
    ----------
    altitude_m : float
        Flight altitude in meters
    focal_length_mm : float
        Lens focal length in millimeters
    pixel_pitch_um : float
        Sensor pixel pitch in micrometers

    Returns
    -------
    dict
        Dictionary with GSD metrics
    """
    # Convert units to meters
    h = altitude_m
    f = focal_length_mm * 1e-3
    p = pixel_pitch_um * 1e-6

    # GSD calculation
    gsd_m = (h * p) / f
    gsd_cm = gsd_m * 100

    # Interpret the GSD
    if gsd_cm < 1:
        interpretation = "Excellent - Can detect small cracks, vegetation details, wires"
        detectable = "License plates, individual bricks, small debris"
    elif gsd_cm < 5:
        interpretation = "Good - Can detect people, cars, building features"
        detectable = "Pedestrians, vehicles, road markings, windows"
    elif gsd_cm < 15:
        interpretation = "Moderate - Can detect vehicles, large features"
        detectable = "Cars, trucks, buildings, roads, field boundaries"
    elif gsd_cm < 50:
        interpretation = "Low - Suitable for large area mapping"
        detectable = "Buildings, roads, large vehicles, land parcels"
    else:
        interpretation = "Very low - Overview/reconnaissance only"
        detectable = "Large structures, terrain features, land use patterns"

    return {
        "gsd_m": gsd_m,
        "gsd_cm": gsd_cm,
        "gsd_mm": gsd_m * 1000,
        "interpretation": interpretation,
        "detectable": detectable,
    }


def display_gsd(altitude_m: float, focal_length_mm: float, pixel_pitch_um: float):
    """Display GSD calculation results."""
    result = calculate_gsd(altitude_m, focal_length_mm, pixel_pitch_um)

    print(f"\n{'=' * 65}")
    print("  GROUND SAMPLING DISTANCE (GSD) CALCULATION")
    print(f"{'=' * 65}")
    print("  Input:")
    print(f"    Altitude (H): {altitude_m:.0f} m")
    print(f"    Focal Length (f): {focal_length_mm:.0f} mm")
    print(f"    Pixel Pitch (p): {pixel_pitch_um:.1f} µm")
    print("")
    print("  Calculation:")
    print("    GSD = H × p / f")
    print(f"    GSD = {altitude_m:.0f} × {pixel_pitch_um:.1f}µm / {focal_length_mm:.0f}mm")
    print(f"    GSD = {result['gsd_cm']:.2f} cm = {result['gsd_mm']:.1f} mm")
    print("")
    print("  Quality Assessment:")
    print(f"    → {result['interpretation']}")
    print(f"    → Detectable: {result['detectable']}")
    print(f"{'=' * 65}\n")


# Interactive widget version
if WIDGETS_AVAILABLE:
    altitude_slider = widgets.FloatSlider(
        value=50,
        min=10,
        max=200,
        step=5,
        description="Altitude (m):",
        continuous_update=False,
        style={"description_width": "100px"},
    )
    focal_slider = widgets.FloatSlider(
        value=50,
        min=20,
        max=100,
        step=5,
        description="Focal (mm):",
        continuous_update=False,
        style={"description_width": "100px"},
    )
    pixel_slider = widgets.FloatSlider(
        value=6.5,
        min=1.5,
        max=10.0,
        step=0.5,
        description="Pixel (µm):",
        continuous_update=False,
        style={"description_width": "100px"},
    )

    output = widgets.interactive_output(
        display_gsd,
        {
            "altitude_m": altitude_slider,
            "focal_length_mm": focal_slider,
            "pixel_pitch_um": pixel_slider,
        },
    )

    display(
        widgets.VBox(
            [
                widgets.HTML("<h4>Adjust parameters:</h4>"),
                altitude_slider,
                focal_slider,
                pixel_slider,
                output,
            ]
        )
    )
else:
    # Static display for environments without widgets
    print("Example calculation:")
    display_gsd(50, 50, 6.5)  # Default values

### Explore Different Scenarios

Try these combinations to build intuition:

In [ ]:
# Common sensor pixel pitches
sensors = {
    'DJI Phantom (1/2.3")': 1.6,
    "1-inch (Sony RX100)": 2.4,
    "APS-C (Canon 80D)": 3.9,
    "Full Frame (Sony A7R)": 6.5,
}

# Common focal lengths
lenses = {"24mm": 24, "35mm": 35, "50mm": 50, "85mm": 85}

# Typical drone altitudes
altitude = 50  # meters

# Create comparison table
print(f"GSD (cm) at {altitude}m altitude for different sensor/lens combinations:")
print("=" * 75)
print(f"{'Sensor':<25}", end="")
for lens_name in lenses:
    print(f"{lens_name:^12}", end="")
print()
print("-" * 75)

for sensor_name, pixel_pitch in sensors.items():
    print(f"{sensor_name:<25}", end="")
    for lens_name, focal_mm in lenses.items():
        result = calculate_gsd(altitude, focal_mm, pixel_pitch)
        print(f"{result['gsd_cm']:^12.2f}", end="")
    print()

print("=" * 75)
print("\nKey insights:")
print("  - Smaller pixels (DJI) can achieve similar GSD to larger pixels at same altitude")
print("  - Longer focal lengths dramatically improve GSD (but reduce coverage)")
print("  - Full-frame sensors need longer lenses to match smaller sensor GSD")

---

# Section 2: Mission Planning with GSD

## Finding the Right Altitude

Often you have a **target GSD** requirement and need to find the correct altitude. Rearranging the formula:

$$H = \frac{\text{GSD} \times f}{p}$$

### Common GSD Requirements

| Application | Required GSD | Typical Altitude |
|-------------|--------------|------------------|
| Infrastructure inspection | 0.5-2 cm | 10-30m |
| Precision agriculture | 2-5 cm | 30-60m |
| Construction monitoring | 3-5 cm | 40-70m |
| Land surveying | 5-10 cm | 50-100m |
| Large area mapping | 10-30 cm | 80-150m |

In [ ]:
def find_altitude_for_gsd(
    target_gsd_cm: float, focal_length_mm: float, pixel_pitch_um: float
) -> float:
    """
    Calculate required altitude for target GSD.

    Parameters
    ----------
    target_gsd_cm : float
        Desired GSD in centimeters
    focal_length_mm : float
        Lens focal length in millimeters
    pixel_pitch_um : float
        Sensor pixel pitch in micrometers

    Returns
    -------
    float
        Required altitude in meters
    """
    gsd_m = target_gsd_cm / 100
    f_m = focal_length_mm * 1e-3
    p_m = pixel_pitch_um * 1e-6

    altitude_m = (gsd_m * f_m) / p_m
    return altitude_m


# Example: Find altitude for 5cm GSD with different lenses
target_gsd = 5  # cm
pixel_pitch = 6.5  # Full-frame sensor

print(f"Required altitude for {target_gsd} cm GSD (Full-frame sensor, {pixel_pitch}µm pixels):")
print("=" * 60)
print(f"{'Lens':<15} {'Altitude':<15} {'Notes':<30}")
print("-" * 60)

lens_options = [
    (24, "Ultra-wide, max coverage"),
    (35, "Wide, good balance"),
    (50, "Standard, versatile"),
    (85, "Telephoto, less coverage"),
]

for focal_mm, notes in lens_options:
    alt = find_altitude_for_gsd(target_gsd, focal_mm, pixel_pitch)
    status = "" if 10 <= alt <= 120 else " (may exceed limits)"
    print(f"{focal_mm}mm{'':<10} {alt:>6.1f} m{status:<8} {notes}")

print("=" * 60)
print("\nNote: Most jurisdictions limit drone altitude to 120m (400ft) AGL")

## Swath Width and Coverage

**Swath width** is the ground coverage in one image dimension:

$$\text{Swath Width} = \frac{H \times \text{Sensor Width}}{f}$$

This determines how much area you cover per pass, which affects:
- Mission flight time
- Number of flight lines needed
- Overlap requirements for photogrammetry

In [ ]:
def calculate_swath(
    altitude_m: float,
    focal_length_mm: float,
    sensor_width_mm: float,
    sensor_height_mm: float,
) -> dict:
    """
    Calculate swath width and coverage area.

    Returns
    -------
    dict
        Dictionary with swath dimensions and area
    """
    f_m = focal_length_mm * 1e-3
    sw_m = sensor_width_mm * 1e-3
    sh_m = sensor_height_mm * 1e-3

    swath_width_m = (altitude_m * sw_m) / f_m
    swath_height_m = (altitude_m * sh_m) / f_m
    coverage_ha = (swath_width_m * swath_height_m) / 10000  # hectares

    return {
        "swath_width_m": swath_width_m,
        "swath_height_m": swath_height_m,
        "coverage_m2": swath_width_m * swath_height_m,
        "coverage_ha": coverage_ha,
    }


# Compare coverage vs detail tradeoff
print("Coverage vs Detail Tradeoff (Full-frame 36x24mm sensor):")
print("=" * 80)
print(f"{'Lens':<10} {'Altitude':<12} {'GSD':<12} {'Swath':<15} {'Coverage':<15} {'Use Case':<20}")
print("-" * 80)

configs = [
    (35, 30, "Inspection"),
    (50, 50, "Survey"),
    (50, 100, "Mapping"),
    (35, 120, "Large area"),
]

for focal_mm, alt_m, use_case in configs:
    gsd = calculate_gsd(alt_m, focal_mm, 6.5)
    swath = calculate_swath(alt_m, focal_mm, 36.0, 24.0)

    print(
        f"{focal_mm}mm{'':<6} {alt_m:>6} m{'':<4} {gsd['gsd_cm']:>6.1f} cm{'':<4} "
        f"{swath['swath_width_m']:>5.0f} x {swath['swath_height_m']:<5.0f}m "
        f"{swath['coverage_ha']:>6.2f} ha{'':<6} {use_case}"
    )

print("=" * 80)
print("\nKey insight: Higher altitude = more coverage but coarser detail")

## Photogrammetry Overlap Requirements

For 3D reconstruction and orthomosaic generation, images must overlap:

- **Front overlap (along track)**: 60-80%
- **Side overlap (between tracks)**: 60-70%

Higher overlap = better reconstruction but more images = longer flights

In [ ]:
# Visualize overlap concept
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Front overlap visualization
ax1 = axes[0]
swath_len = 100  # arbitrary units
overlaps = [0.6, 0.7, 0.8]  # 60%, 70%, 80%
colors = ["#3498db", "#2ecc71", "#e74c3c"]

for i, (overlap, color) in enumerate(zip(overlaps, colors)):
    spacing = swath_len * (1 - overlap)
    y_offset = i * 30

    # Draw three consecutive images
    for j in range(3):
        x_start = j * spacing
        rect = plt.Rectangle(
            (x_start, y_offset),
            swath_len,
            20,
            fill=True,
            alpha=0.3,
            edgecolor=color,
            facecolor=color,
            linewidth=2,
        )
        ax1.add_patch(rect)

    ax1.text(
        -15, y_offset + 10, f"{int(overlap * 100)}%", va="center", fontsize=11, fontweight="bold"
    )

ax1.set_xlim(-30, 200)
ax1.set_ylim(-10, 100)
ax1.set_title("Front Overlap (Along Track)", fontsize=14)
ax1.set_xlabel("Flight Direction →")
ax1.set_aspect("equal")
ax1.axis("off")

# Coverage diagram
ax2 = axes[1]

# Draw a grid of overlapping images
front_overlap = 0.7
side_overlap = 0.6
img_width = 100
img_height = 67  # 3:2 aspect ratio

x_spacing = img_width * (1 - side_overlap)
y_spacing = img_height * (1 - front_overlap)

for row in range(3):
    for col in range(4):
        x = col * x_spacing
        y = row * y_spacing
        rect = plt.Rectangle(
            (x, y),
            img_width,
            img_height,
            fill=True,
            alpha=0.2,
            edgecolor="blue",
            facecolor="blue",
            linewidth=1,
        )
        ax2.add_patch(rect)

ax2.set_xlim(-20, 280)
ax2.set_ylim(-20, 160)
ax2.set_title(
    f"Flight Pattern: {int(front_overlap * 100)}% front, {int(side_overlap * 100)}% side overlap",
    fontsize=14,
)
ax2.set_xlabel("Cross-track →")
ax2.set_ylabel("Along-track →")
ax2.set_aspect("equal")
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nTypical overlap recommendations:")
print("  - Flat terrain: 60% front, 60% side")
print("  - Rolling terrain: 70% front, 65% side")
print("  - Complex 3D structures: 80% front, 70% side")

## Lens Comparison: 35mm vs 50mm vs 85mm

Let's visualize how lens choice affects GSD across different altitudes:

In [ ]:
# GSD vs Altitude curves for different lenses
altitudes = np.linspace(10, 150, 100)
pixel_pitch = 6.5  # Full-frame

lenses_to_compare = [
    ("35mm", 35, "#e74c3c"),
    ("50mm", 50, "#2ecc71"),
    ("85mm", 85, "#3498db"),
]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: GSD vs Altitude
for name, focal_mm, color in lenses_to_compare:
    gsds = [calculate_gsd(alt, focal_mm, pixel_pitch)["gsd_cm"] for alt in altitudes]
    ax1.plot(altitudes, gsds, label=name, color=color, linewidth=2)

# Add reference lines for common GSD requirements
gsd_refs = [(5, "5cm - Inspection"), (10, "10cm - Survey"), (20, "20cm - Mapping")]
for gsd_val, label in gsd_refs:
    ax1.axhline(y=gsd_val, color="gray", linestyle="--", alpha=0.5)
    ax1.text(152, gsd_val, label, va="center", fontsize=9, color="gray")

ax1.set_xlabel("Altitude (m)", fontsize=12)
ax1.set_ylabel("GSD (cm)", fontsize=12)
ax1.set_title(
    "GSD vs Altitude for Different Lenses\n(Full-frame sensor, 6.5µm pixels)", fontsize=14
)
ax1.legend(loc="upper left")
ax1.grid(True, alpha=0.3)
ax1.set_xlim(10, 150)
ax1.set_ylim(0, 30)

# Plot 2: Swath Width vs Altitude
for name, focal_mm, color in lenses_to_compare:
    swaths = [calculate_swath(alt, focal_mm, 36, 24)["swath_width_m"] for alt in altitudes]
    ax2.plot(altitudes, swaths, label=name, color=color, linewidth=2)

ax2.set_xlabel("Altitude (m)", fontsize=12)
ax2.set_ylabel("Swath Width (m)", fontsize=12)
ax2.set_title("Swath Width vs Altitude\n(Coverage per image)", fontsize=14)
ax2.legend(loc="upper left")
ax2.grid(True, alpha=0.3)
ax2.set_xlim(10, 150)

plt.tight_layout()
plt.show()

print("\nLens selection guide:")
print("  - 35mm: Maximum coverage, higher GSD - best for large area mapping")
print("  - 50mm: Balanced choice - good for general surveying")
print("  - 85mm: Best GSD at same altitude - ideal for detail inspection")

---

# Section 3: Diffraction Limits vs GSD

## Does Diffraction Matter for Drones?

In microscopy, diffraction is the fundamental limit. But for drone imaging, the situation is different.

### The Airy Disk at the Sensor

Diffraction causes a point source to form an **Airy disk** at the sensor:

$$d_{\text{Airy}} = 2.44 \times \lambda \times (f/\#)$$

Where:
- $d_{\text{Airy}}$ = Airy disk diameter at sensor
- $\lambda$ = Wavelength of light
- $f/\#$ = f-number (focal ratio)

### When Does Diffraction Matter?

- **Pixel-limited**: If Airy disk < pixel size → GSD determines resolution
- **Diffraction-limited**: If Airy disk > pixel size → Diffraction determines resolution

For typical drone cameras (f/2.8 to f/8), the Airy disk is **much smaller** than the pixel size!

In [ ]:
def calculate_airy_disk(wavelength_nm: float, f_number: float) -> float:
    """
    Calculate Airy disk diameter at sensor.

    Parameters
    ----------
    wavelength_nm : float
        Wavelength in nanometers
    f_number : float
        f-number (focal ratio)

    Returns
    -------
    float
        Airy disk diameter in micrometers
    """
    wavelength_um = wavelength_nm * 1e-3
    airy_diameter_um = 2.44 * wavelength_um * f_number
    return airy_diameter_um


# Compare Airy disk to common pixel sizes
wavelength = 550  # nm (green light)
f_numbers = [2.0, 2.8, 4.0, 5.6, 8.0, 11.0, 16.0]

print("Airy Disk Diameter vs Pixel Size (λ = 550nm):")
print("=" * 75)
print(f"{'f-number':<10} {'Airy Disk':<15} {'vs 6.5µm':<15} {'vs 3.9µm':<15} {'vs 1.6µm':<15}")
print("-" * 75)

for f_num in f_numbers:
    airy = calculate_airy_disk(wavelength, f_num)

    # Compare to common pixel sizes
    def status(airy, pixel):
        if airy < pixel * 0.5:
            return "Pixel-limited"
        elif airy < pixel:
            return "Balanced"
        else:
            return "DIFFRACTION!"

    print(
        f"f/{f_num:<7} {airy:>6.2f} µm{'':<6} {status(airy, 6.5):<15} "
        f"{status(airy, 3.9):<15} {status(airy, 1.6):<15}"
    )

print("=" * 75)
print("\nKey insight: At f/4 or wider, even small-pixel sensors are pixel-limited")
print("Diffraction only matters at f/8+ with small pixels (< 2µm)")

In [ ]:
# Visualize the comparison
fig, ax = plt.subplots(figsize=(10, 6))

f_nums = np.linspace(1.4, 22, 100)
airy_diameters = [calculate_airy_disk(550, f) for f in f_nums]

ax.plot(f_nums, airy_diameters, "b-", linewidth=2, label="Airy disk diameter")

# Reference lines for pixel sizes
pixel_sizes = [
    (6.5, "Full-frame (6.5µm)", "#27ae60"),
    (3.9, "APS-C (3.9µm)", "#f39c12"),
    (1.6, "DJI Phantom (1.6µm)", "#e74c3c"),
]

for pixel, label, color in pixel_sizes:
    ax.axhline(y=pixel, color=color, linestyle="--", linewidth=2, label=label)

# Shade regions
ax.fill_between(f_nums, 0, airy_diameters, alpha=0.1, color="blue")

# Mark typical drone f-numbers
ax.axvline(x=2.8, color="gray", linestyle=":", alpha=0.7)
ax.axvline(x=4.0, color="gray", linestyle=":", alpha=0.7)
ax.text(2.8, 22, "f/2.8", ha="center", fontsize=10)
ax.text(4.0, 22, "f/4.0", ha="center", fontsize=10)

ax.set_xlabel("f-number", fontsize=12)
ax.set_ylabel("Size (µm)", fontsize=12)
ax.set_title("Airy Disk vs Pixel Size: When Does Diffraction Limit Resolution?", fontsize=14)
ax.legend(loc="upper left")
ax.grid(True, alpha=0.3)
ax.set_xlim(1.4, 22)
ax.set_ylim(0, 25)

plt.tight_layout()
plt.show()

print("\nConclusion for drone imaging:")
print("  - At typical f/2.8-f/4, ALL sensors are pixel-limited")
print("  - GSD (not diffraction) determines your practical resolution")
print("  - Diffraction only becomes relevant if you stop down to f/11+")

## The Fresnel Number: Propagation Regime

The **Fresnel number** indicates which optical regime applies:

$$F = \frac{a^2}{\lambda z}$$

Where:
- $a$ = aperture radius
- $\lambda$ = wavelength
- $z$ = propagation distance (altitude)

**Interpretation:**
- $F >> 1$: Geometric optics (ray tracing valid)
- $F \sim 1$: Fresnel diffraction (need wave optics)
- $F << 1$: Fraunhofer diffraction (far-field)

In [ ]:
def calculate_fresnel_number(
    aperture_diameter_mm: float, wavelength_nm: float, distance_m: float
) -> float:
    """
    Calculate Fresnel number.

    Parameters
    ----------
    aperture_diameter_mm : float
        Aperture diameter in millimeters
    wavelength_nm : float
        Wavelength in nanometers
    distance_m : float
        Propagation distance in meters

    Returns
    -------
    float
        Fresnel number
    """
    a = (aperture_diameter_mm * 1e-3) / 2  # radius in meters
    wavelength = wavelength_nm * 1e-9  # meters
    return (a**2) / (wavelength * distance_m)


# Calculate Fresnel number for drone scenarios
print("Fresnel Number for Drone Imaging Scenarios:")
print("=" * 75)
print(f"{'Scenario':<25} {'Altitude':<10} {'Aperture':<12} {'Fresnel #':<12} {'Regime':<15}")
print("-" * 75)

scenarios = [
    ("Close inspection", 10, 50 / 4.0),  # 50mm f/4
    ("Standard survey", 50, 50 / 4.0),
    ("High altitude", 100, 50 / 4.0),
    ("Max altitude", 200, 50 / 4.0),
    ("Wide lens f/2.8", 50, 35 / 2.8),
    ("Telephoto f/4", 50, 85 / 4.0),
]

for name, altitude, aperture_mm in scenarios:
    fresnel = calculate_fresnel_number(aperture_mm, 550, altitude)

    if fresnel > 100:
        regime = "Geometric"
    elif fresnel > 10:
        regime = "Near-geometric"
    elif fresnel > 1:
        regime = "Fresnel"
    else:
        regime = "Fraunhofer"

    print(
        f"{name:<25} {altitude:>5} m{'':<3} {aperture_mm:>6.1f} mm{'':<4} {fresnel:>8.0f}{'':<4} {regime}"
    )

print("=" * 75)
print("\nFor drone imaging (F >> 1), geometric optics is valid.")
print("GSD formula is accurate; no need for wave propagation calculations.")

---

# Assessment Questions

Test your understanding with these questions:

In [ ]:
print("=" * 70)
print("ASSESSMENT QUESTIONS")
print("=" * 70)

print("\nQuestion 1:")
print("   Calculate GSD for: 50mm lens, 100m altitude, 4µm pixels")
print("")
print("   Your calculation:")
q1_result = calculate_gsd(100, 50, 4.0)
print(f"   GSD = H × p / f = 100 × 4µm / 50mm = {q1_result['gsd_cm']:.1f} cm")
print("   Expected: 8.0 cm")

print("\n" + "-" * 70)

print("\nQuestion 2:")
print("   Which lens for 5cm GSD at 50m altitude with 6.5µm pixels?")
print("")
print("   Analysis:")
print("   Required focal length: f = H × p / GSD")
print("   f = 50m × 6.5µm / 0.05m = 6.5mm... that's wrong!")
print("   Let's recalculate: f = (50 × 6.5e-6) / 0.05 = 0.0065m = 6.5mm")
print("")
print("   Wait - that would need a 6.5mm lens (fisheye territory)!")
print("   Let's check what GSD we get with common lenses at 50m:")
for focal in [35, 50, 85]:
    gsd = calculate_gsd(50, focal, 6.5)["gsd_cm"]
    print(f"     {focal}mm → {gsd:.1f} cm")
print("")
print("   Answer: Need ~65mm lens, or fly lower! 35mm at 27m would work.")
alt_needed = find_altitude_for_gsd(5, 35, 6.5)
print(f"   (35mm lens at {alt_needed:.0f}m gives 5cm GSD)")

print("\n" + "-" * 70)

print("\nQuestion 3:")
print("   Why does diffraction usually NOT matter for drone imaging?")
print("")
print("   Answer:")
print("   - At f/4, Airy disk ≈ 5.4µm (smaller than most pixels)")
print("   - GSD at 50m with 6.5µm pixels ≈ 6.5cm (MUCH larger than Airy disk)")
print("   - The ground resolution is limited by GSD, not diffraction")
print("   - Fresnel number >> 1 means geometric optics applies")
print("")
print("   Key insight: GSD >> diffraction blur, so GSD dominates!")

print("\n" + "=" * 70)

---

# Bonus: SPIDS Drone Presets

Let's explore the 8 drone presets available in SPIDS:

In [ ]:
# List all drone presets
drone_presets = list_scenario_presets("drone")

print("SPIDS Drone Camera Presets:")
print("=" * 95)
print(f"{'Preset':<28} {'Altitude':<10} {'Lens':<12} {'Sensor':<15} {'GSD':<10} {'Swath':<10}")
print("-" * 95)

for preset_name in drone_presets:
    scenario = get_scenario_preset(preset_name)

    # Get info
    info = scenario.get_info()

    print(
        f"{preset_name:<28} {info['altitude_m']:>5.0f} m{'':<3} "
        f"{info['lens']:<12} {info['sensor']:<15} "
        f"{info['gsd_cm']:>5.1f} cm{'':<3} {info['swath_width_m']:>5.0f} m"
    )

print("=" * 95)
print(f"\nTotal: {len(drone_presets)} drone presets available")

In [ ]:
# Visualize preset comparison
fig, ax = plt.subplots(figsize=(12, 6))

presets_data = []
for preset_name in drone_presets:
    scenario = get_scenario_preset(preset_name)
    info = scenario.get_info()
    presets_data.append(
        {
            "name": preset_name.replace("drone_", ""),
            "altitude": info["altitude_m"],
            "gsd": info["gsd_cm"],
            "swath": info["swath_width_m"],
        }
    )

# Scatter plot: GSD vs Swath (size = altitude)
gsds = [p["gsd"] for p in presets_data]
swaths = [p["swath"] for p in presets_data]
altitudes = [p["altitude"] for p in presets_data]
names = [p["name"] for p in presets_data]

scatter = ax.scatter(
    gsds,
    swaths,
    s=[a * 3 for a in altitudes],
    c=altitudes,
    cmap="viridis",
    alpha=0.7,
    edgecolors="black",
)

# Label each point
for i, name in enumerate(names):
    ax.annotate(
        name,
        (gsds[i], swaths[i]),
        xytext=(5, 5),
        textcoords="offset points",
        fontsize=9,
    )

ax.set_xlabel("GSD (cm)", fontsize=12)
ax.set_ylabel("Swath Width (m)", fontsize=12)
ax.set_title("SPIDS Drone Presets: Detail vs Coverage\n(bubble size = altitude)", fontsize=14)

cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label("Altitude (m)", fontsize=11)

ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nUse case guide:")
print("  - Lower-left (low GSD, low swath): Detailed inspection")
print("  - Upper-right (high GSD, high swath): Large area coverage")
print("  - Choose based on your mission requirements!")

---

# Summary

## Key Takeaways

### 1. The GSD Formula
$$\text{GSD} = \frac{H \times p}{f}$$
- Higher altitude → Larger GSD (less detail)
- Smaller pixels → Smaller GSD (more detail)
- Longer focal length → Smaller GSD (more "zoom")

### 2. Mission Planning
- Start with your required GSD, then calculate altitude
- Consider swath width for coverage planning
- Use 60-80% overlap for photogrammetry

### 3. Lens Selection
- Wide lenses (24-35mm): Maximum coverage, higher GSD
- Standard lenses (50mm): Good balance
- Telephoto (85mm+): Best detail, less coverage

### 4. Diffraction in Drone Imaging
- At typical f/2.8-f/4, all sensors are **pixel-limited**
- GSD >> diffraction blur, so GSD dominates
- Fresnel number >> 1 means geometric optics applies

## Quick Reference

| GSD | Detectable Features | Typical Use |
|-----|---------------------|-------------|
| < 1 cm | Cracks, small defects | Infrastructure inspection |
| 1-5 cm | People, vehicles, signs | Site monitoring |
| 5-15 cm | Buildings, roads | Surveying, agriculture |
| 15-50 cm | Land parcels, terrain | Large area mapping |

---

## Reference Documentation

For detailed parameter lookup and formulas:

- **[Scenario Preset Catalog](../../docs/references/scenario_preset_catalog.md)** - All 8 drone presets
- **[Drone Camera Configuration](../../docs/references/drone_camera_configuration.md)** - Sensor and lens specs

---

## Next Steps

- **[Learning 5: Drone Altitude Comparison](learning_05_drone_altitudes.ipynb)**: Compare all 8 presets
- **[Python API: Drone Mapping](../python_api/07_drone_mapping.py)**: Full reconstruction workflow
- **[Validation: GSD Validation](../validation/notebooks/04_drone_gsd_validation.ipynb)**: Verify GSD accuracy

In [ ]:
print("\n" + "=" * 70)
print("Congratulations! You've completed Learning 4: GSD Fundamentals")
print("=" * 70)
print("\nYou now understand:")
print("  - The GSD formula and how to calculate it")
print("  - How to plan missions with specific GSD requirements")
print("  - The tradeoff between coverage and detail")
print("  - Why diffraction usually doesn't matter for drones")
print("  - How to use SPIDS drone presets")
print("\nReady for Learning 5? Compare drone presets at different altitudes!")